In [2]:
import os
from tensorflow import keras
from keras.models import Sequential

from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tqdm import tqdm
from keras.layers import BatchNormalization

import cv2

In [2]:
# Set the image directory and read metadata
image_directory = 'dataset_multilabel/train'
metadata_path = os.path.join(image_directory, '_classes.csv')

# Load metadata into a DataFrame
data = pd.read_csv(metadata_path)
data.head()

,filename,Unlabeled,crossbite,normal,openbite,overbite,underbite
0,crossbite-55_png.rf.cb478819018a0965c6101b65c6...,0,1,0,0,0,0
1,openbite-57_png.rf.cb92146b66b084917d2a2dd8ea4...,0,0,0,1,0,0
2,underbite-19_png.rf.cbe70ee74f3e6cc63ed7d9c2ea...,0,1,0,0,0,1
3,crossbite-12_jpeg.rf.cc19da26efe5e417325030b8a...,0,1,0,0,0,0
4,openbite-38_jpeg.rf.cc21ca0d5ac758e309b6464481...,0,0,0,1,0,0


In [3]:
# Preprocessing function to load and preprocess images
def load_and_preprocess_image(filename):
    # Construct the full image path
    image_path = os.path.join(image_directory, filename)
    
    # Load and preprocess the image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
    image = cv2.resize(image, (150, 150))  # Resize
    image = image / 255.0  # Normalize pixel values to [0, 1]
    return image

In [4]:
# Apply the preprocessing function to all images in the DataFrame
image = data['filename'].apply(load_and_preprocess_image).to_list()

In [5]:
X = np.array(image)
y = np.array(data.drop(['filename'], axis=1))

In [6]:
# First, split the data into training and remaining using 70% for training
X_train, X_remaining, y_train, y_remaining = train_test_split(X, y, test_size=0.3, random_state=42)

# Then split the remaining data into testing and validation sets (15% each of the original data)
X_test, X_val, y_test, y_val = train_test_split(X_remaining, y_remaining, test_size=0.5, random_state=42)

In [7]:
# Load the pre-trained VGG16 model
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))

# Set the pre-trained layers as non-trainable
for layer in vgg16_model.layers:
    layer.trainable = True

# Build the model by adding the VGG16 base and additional layers
model = keras.Sequential([
    vgg16_model,
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(6, activation='sigmoid')
])

2023-09-16 13:43:28.873369: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-16 13:43:28.929184: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-16 13:43:28.929320: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-16 13:43:28.930092: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the approp

In [8]:
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), batch_size=16)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 64)                524352    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 6)                 390       
                                                                 
Total params: 15,239,430
Trainable params: 524,742
Non-trainable params: 14,714,688
_________________________________________________________________
Epoch 1/50


2023-09-16 13:43:31.638360: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8201
2023-09-16 13:43:33.958464: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.54GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


42/42 [==============================] - ETA: 0s - loss: 0.4122 - accuracy: 0.4331

2023-09-16 13:43:41.414129: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.50GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


42/42 [==============================] - 12s 181ms/step - loss: 0.4122 - accuracy: 0.4331 - val_loss: 0.2988 - val_accuracy: 0.7203
Epoch 2/50
42/42 [==============================] - 5s 117ms/step - loss: 0.2902 - accuracy: 0.6316 - val_loss: 0.2590 - val_accuracy: 0.6503
Epoch 3/50
42/42 [==============================] - 5s 114ms/step - loss: 0.2341 - accuracy: 0.7233 - val_loss: 0.2423 - val_accuracy: 0.6573
Epoch 4/50
42/42 [==============================] - 5s 113ms/step - loss: 0.1941 - accuracy: 0.7850 - val_loss: 0.2241 - val_accuracy: 0.7063
Epoch 5/50
42/42 [==============================] - 5s 109ms/step - loss: 0.1865 - accuracy: 0.7940 - val_loss: 0.2018 - val_accuracy: 0.7622
Epoch 6/50
42/42 [==============================] - 5s 114ms/step - loss: 0.1605 - accuracy: 0.8361 - val_loss: 0.2063 - val_accuracy: 0.7972
Epoch 7/50
42/42 [==============================] - 5s 118ms/step - loss: 0.1394 - accuracy: 0.8526 - val_loss: 0.1868 - val_accuracy: 0.8112
Epoch 8/50
42/42

In [9]:
from sklearn.metrics import classification_report

# Make predictions on the test data
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

# Generate the classification report
cr_ann = classification_report(y_test, y_pred, target_names=['Unlabeled', 'crossbite', 'normal', 'openbite', 'overbite', 'underbite'])

# Print the classification report
print(cr_ann)

2023-09-16 13:47:35.085378: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2023-09-16 13:47:35.533127: W tensorflow/core/common_runtime/bfc_allocator.cc:290] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


5/5 [==============================] - 2s 134ms/step
              precision    recall  f1-score   support

   Unlabeled       0.00      0.00      0.00         0
   crossbite       0.89      0.62      0.73        39
      normal       0.89      0.89      0.89        28
    openbite       1.00      1.00      1.00        27
    overbite       0.88      0.86      0.87        50
   underbite       1.00      0.17      0.29         6

   micro avg       0.91      0.80      0.85       150
   macro avg       0.78      0.59      0.63       150
weighted avg       0.91      0.80      0.84       150
 samples avg       0.83      0.83      0.83       150



/home/fadhel/Workspaces/Project Intern/dental-disease-detection-classification/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fadhel/Workspaces/Project Intern/dental-disease-detection-classification/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/fadhel/Workspaces/Project Intern/dental-disease-detection-classification/.venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 i

In [10]:
# Save the model to a file
model.save("classification.h5")